# QMMM workflow using GROMACS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use methane as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://www.votca.org/xtp/xtp_map.html)
* [xtp_parallel](https://www.votca.org/xtp/xtp_parallel.html)
* [xtp_run](https://www.votca.org/xtp/xtp_run.html)
* [xtp_tools](https://www.votca.org/xtp/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          Tool options
  -t [ --nthreads ] arg (=1)    number of threads to create
  -e [ --execute ] arg        Name of Tool to run
  -l [ --list ]               Lists all available Tools
  -d [ --description ] arg    Short description of a Tools
  -c [ --cmdoptions ] arg     Modify options via command line by e.g. '-c 
                              x

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [2]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [3]:
!rm -f state.hdf5

Import an small python module to edit xml files

In [4]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file

runs the mapping from MD coordinates to segments and creates an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/). You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/).

In [5]:
!xtp_map -t MD_FILES/topol.tpr -c MD_FILES/conf.gro -s system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_map, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Reading file MD_FILES/topol.tpr, VERSION 5.1.1 (single precision)
Note: file tpx version 103, software tpx version 116
Creating statefile state.hdf5
Frame with id 0 was not in statefile state.hdf5 ,adding it now.
Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. We will need to copy the input from the `VOTCA` installation path to our local copy.

In [6]:
!cp "$VOTCASHARE/xtp/xml/mapchecker.xml" OPTIONFILES/

The `${VOTCASHARE}` environmental variable is set to the path that you provided during the `VOTCA` [installation](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst), by the default is set to `/usr/local/votca`.

We need to update the `mapchecker` options with the file containing the definition of the system. For doing, so we will use the python module `xml_editor` that we have imported above. The first argument is the name of the calculator (i.e. `mapchecker`), the seconds argument is the name of the option to change and the third one, the value to use.

In [7]:
edit_calculator("mapchecker", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/mapchecker.xml' has been set to 'system.xml'


In the [mapchecker section of the manual](https://votca.github.io/xtp/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [8]:
!xtp_run -e mapchecker -o OPTIONFILES/mapchecker.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... mapchecker
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... mapchecker
 Using 1 threads
Writing segments to md_segments step__0.pdb
Writing qmmolecules to qm_segments_n step__0.pdb
Writing polarsegments to mp_segments_e step__0.pdb
Writing polarsegments to mp_segments_h step__0.pdb
Changes have not been written to state file.


## Neighborlist Calculation

The following step is to determine the neighbouring pairs for exciton transport. We will need first to get a copy of the input 

In [9]:
cp "$VOTCASHARE/xtp/xml/neighborlist.xml" OPTIONFILES/

Then we will use `segmentname` method to select the cutoff and set the *exciton_cutoff* for the pairs to 0.5 nanometers. See the [neighborlist options](https://www.votca.org/xtp/neighborlist.html) for further information. 

In [10]:
exciton_cutoff = """
<exciton_cutoff>0.5</exciton_cutoff>
"""
segments = """
<segments>
  <type>Methane Methane</type>
  <cutoff>1</cutoff>
</segments>
"""
constant = """<constant help="cutoff for all other types of pairs">0.6</constant>"""
cutoff_type = """<cutoff_type>segmentname</cutoff_type>"""

add_section("neighborlist", cutoff_type)
add_section("neighborlist", exciton_cutoff)
add_section("neighborlist", segments)
add_section("neighborlist", constant)

Finally, we can run the calculation using 4 threads

In [11]:
!xtp_run -e neighborlist -o OPTIONFILES/neighborlist.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... neighborlist
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... neighborlist
 Using 1 threads
Evaluating 1000 segments for neighborlist. 
The following segments are used in the neigborlist creation
	 Methane
 ... ... Evaluating 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

 ... ... Created 76140 direct pairs.Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation

In [12]:
!cp "$VOTCASHARE/xtp/xml/einternal.xml" OPTIONFILES/
!xtp_run -e einternal -o OPTIONFILES/einternal.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... einternal
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... einternal
 Using 1 threads
... ... Site, reorg. energies from system.xml.

... ... Read in site, reorg. energies for 1000 segments. Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file,

In [13]:
!cp qmmm_mm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
... ... Writing job file qmmm_mm_jobs.xml
... ... In total 4001 jobs
Changes have not been written to state file.


The previous command generates a `qmmm_mm_jobs.xml` containing 4000 *MM* jobs to compute, if you examine that file, it should look something like

```xml
<jobs>
  <job>
    <id>0</id>
    <tag>Methane_0:n</tag>
    <input>
      <site_energies>0:n</site_energies>
      <regions>
        <region>
          <id>0</id>
          <segments>0:n</segments>
        </region>
      </regions>
    </input>
    <status>AVAILABLE</status>
  </job>    
...
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [14]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [15]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -x 2 -j "run"
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
MST ERR Job file = 'qmmm_mm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_mm_jobs.xml
MST ERR Registered 4000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ...  Regions created
T00 ERR ... Id: 0 type: polar size: 17 charge[e]= -9.29812e-16
T00 ERR ... Id: 1 type: static size: 254 charge[e]= -1.4086e

## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [16]:
!cp "$VOTCASHARE/xtp/xml/eanalyze.xml" OPTIONFILES/
!xtp_run -e eanalyze -o OPTIONFILES/eanalyze.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... eanalyze
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eanalyze
 Using 1 threads
... ... Short-listed 1000 segments (pattern='*')
... ... ... NOTE Statistics of site energies and spatial correlations thereof are based on the short-listed segments only. 
... ... ...      Statistics of site-energy differences operate on the full list.
... ... excited state e
... ... excited state h
... ... excited state s
... ... excited state tChanges have not been written to state file.


You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [17]:
!ls eanalyze*

eanalyze.pairhist_e.out  eanalyze.pairlist_s.out  eanalyze.sitehist_e.out
eanalyze.pairhist_h.out  eanalyze.pairlist_t.out  eanalyze.sitehist_h.out
eanalyze.pairhist_s.out  eanalyze.sitecorr_e.out  eanalyze.sitehist_s.out
eanalyze.pairhist_t.out  eanalyze.sitecorr_h.out  eanalyze.sitehist_t.out
eanalyze.pairlist_e.out  eanalyze.sitecorr_s.out
eanalyze.pairlist_h.out  eanalyze.sitecorr_t.out


## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We will first copy the input into our local folder

In [18]:
!cp "$VOTCASHARE/xtp/xml/eqm.xml" OPTIONFILES/

Now we set the GWBSE mode to `G0W0`,  the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. For more information, check the [eqm calculator options](https://votca.github.io/xtp/eqm.html).

In [19]:
edit_calculator("eqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("eqm", "gwbse_options.gwbse.mode", "G0W0")
edit_calculator("eqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("eqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("eqm", "map_file", "system.xml")
edit_calculator("eqm", "basisset", "3-21G")
edit_calculator("eqm", "auxbasisset", "aux-def2-svp")

The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/eqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.mode' on file 'OPTIONFILES/eqm.xml' has been set to 'G0W0'
The option 'gwbse_options.gwbse.qp_solver' on file 'OPTIONFILES/eqm.xml' has been set to 'fixedpoint'
The option 'gwbse_options.gwbse.qp_grid_steps' on file 'OPTIONFILES/eqm.xml' has been set to '100'
The option 'gwbse_options.gwbse.qp_grid_spacing' on file 'OPTIONFILES/eqm.xml' has been set to '0.01'
The option 'gwbse_options.gwbse.exctotal' on file 'OPTIONFILES/eqm.xml' has been set to '10'
The option 'map_file' on file 'OPTIONFILES/eqm.xml' has been set to 'system.xml'
The option 'basisset' on file 'OPTIONFILES/eqm.xml' has been set to '3-21G'
The option 'auxbasisset' on file 'OPTIONFILES/eqm.xml' has been set to 'aux-def2-svp'


For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://www.votca.org/xtp/eqm.html).

In [20]:
edit_calculator("eqm", "gwbse_options.gwbse.tasks", "gw,singlets")

The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/eqm.xml' has been set to 'gw,singlets'


First we will write the job in a file and enable only the first 2,

In [21]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs


========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... eqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eqm 
... ... Writing job file: eqm.jobs with 1000 jobs
Changes have not been written to state file.


Now, let run these 2 jobs

In [22]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... eqm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eqm 
MST ERR Job file = 'eqm.jobs', cache size =  1
MST ERR Initialize jobs from eqm.jobs
MST ERR Registered 1000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2021-7-7 16:41:36 Evaluating site 0
T00 ERR ... Running DFT
T00 ERR ... Running GWBSE
T00 ERR ... Running ESPFIT
T00 ERR ... ===== Running on 2 threads ===== 

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

In [23]:
!cp "$VOTCASHARE/xtp/xml/iqm.xml" OPTIONFILES/

As in the previous section, we set the GWBSE mode to `G0W0`, the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. But we compute only the `gw` approximation. We are also going to compute a single excited state. For more information, check the [iqm calculator options](https://www.votca.org/xtp/iqm.html). We also want to compute the `singlet` couplings. 

In [24]:
edit_calculator("iqm", "states", "1")
edit_calculator("iqm", "map_file", "system.xml")
edit_calculator("iqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("iqm", "gwbse_options.gwbse.tasks", "gw")
edit_calculator("iqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("iqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("iqm", "bsecoupling.spin", "singlet")
edit_calculator("iqm", "basisset", "3-21G")
edit_calculator("iqm", "auxbasisset", "aux-def2-svp")

The option 'states' on file 'OPTIONFILES/iqm.xml' has been set to '1'
The option 'map_file' on file 'OPTIONFILES/iqm.xml' has been set to 'system.xml'
The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/iqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/iqm.xml' has been set to 'gw'
The option 'gwbse_options.gwbse.qp_solver' on file 'OPTIONFILES/iqm.xml' has been set to 'fixedpoint'
The option 'gwbse_options.gwbse.qp_grid_steps' on file 'OPTIONFILES/iqm.xml' has been set to '100'
The option 'gwbse_options.gwbse.qp_grid_spacing' on file 'OPTIONFILES/iqm.xml' has been set to '0.01'
The option 'gwbse_options.gwbse.exctotal' on file 'OPTIONFILES/iqm.xml' has been set to '10'
The option 'bsecoupling.spin' on file 'OPTIONFILES/iqm.xml' has been set to 'singlet'
The option 'basisset' on file 'OPTIONFILES/iqm.xml' has been set to '3-21G'
The option 'auxbasisset' on file 'OPTIONFILES/iqm.xml' has been set to 'aux-def2-svp'


Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type. The following *XML* snippet needs to be added

In [25]:
readjobfile = """
<readjobfile help="which states to read into the jobfile for each segment type">
     <singlet>Methane:s1</singlet>
     <triplet>Methane:t1</triplet>
     <electron>Methane:e1</electron>
     <hole>Methane:h1</hole>
</readjobfile>
"""

The following code add the `readjobfile` section,

In [26]:
add_section("iqm", readjobfile)

Now, let's write the jobs to the file

In [27]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j "write"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
... ... Writing job file iqm.jobs
... ... In total 76140 jobs
Changes have not been written to state file.


From the jobs that we just write down, let's make available only the first job

In [28]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [29]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
MST ERR Job file = 'iqm.jobs', cache size =  1
MST ERR Initialize jobs from iqm.jobs
MST ERR Registered 76140 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2021-7-7 16:41:59 Evaluating pair 0 [0:1] out of 76140
T00 ERR ... Guess requested, reading molecular orbitals
T00 ERR ... Reading MoleculeA from OR_FILES/mo

Finally, we read the results into the state

In [30]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
 ERROR    Pairs [total:updated(e,h,s,t)] 76140:(1,1,1,0) Incomplete jobs: 76139

Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Coupling
We can now compute the classical coupling of transition in the aformentioned three stages,

In [31]:
!cp "${VOTCASHARE}/xtp/xml/iexcitoncl.xml" OPTIONFILES

We need to change in the `iexcitoncl` input the name `map_file` option and add the state. check all the available of the [iexcitoncl calculator](https://www.votca.org/xtp/iexcitoncl.html).

In [32]:
edit_calculator("iexcitoncl", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/iexcitoncl.xml' has been set to 'system.xml'


In [33]:
state = "<states>Methane:n2s1</states>"
add_section("iexcitoncl", state)

In [34]:
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... iexcitoncl

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iexcitoncl 
... ... Writing job file exciton.jobs
... ... In total 0 jobs
Changes have not been written to state file.


In [35]:
!ls


 MD_FILES		   eanalyze.pairlist_s.out   iqm.jobs~
 MMMM			   eanalyze.pairlist_t.out  'md_segments step__0.pdb'
 MP_FILES		   eanalyze.sitecorr_e.out  'mp_segments_e step__0.pdb'
 OPTIONFILES		   eanalyze.sitecorr_h.out  'mp_segments_h step__0.pdb'
 OR_FILES		   eanalyze.sitecorr_s.out  'qm_segments_n step__0.pdb'
 QC_FILES		   eanalyze.sitecorr_t.out   qmmm.xml
 QMMM_GROMACS.ipynb	   eanalyze.sitehist_e.out   qmmm_mm.xml
 __pycache__		   eanalyze.sitehist_h.out   qmmm_mm_jobs.xml
 eanalyze.pairhist_e.out   eanalyze.sitehist_s.out   qmmm_mm_jobs.xml~
 eanalyze.pairhist_h.out   eanalyze.sitehist_t.out   state.hdf5
 eanalyze.pairhist_s.out   eqm.jobs		     system.xml
 eanalyze.pairhist_t.out   eqm.jobs~		     xml_editor.py
 eanalyze.pairlist_e.out   exciton.jobs
 eanalyze.pairlist_h.out   iqm.jobs


Now we can run and save the jobs. For demo purposes we will run only the first job

In [36]:
!sed -i "s/AVAILABLE/COMPLETE/g" exciton.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' exciton.jobs
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... iexcitoncl

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iexcitoncl 
MST ERR Job file = 'exciton.jobs', cache size =  8
MST ERR Initialize jobs from exciton.jobs
MST ERR Registered 0 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Next job: ID = - (none available)
MST ERR Assign jobs from stack
Changes have not been written to state file.


In [37]:
#!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "read"

## Coupling analysis
Using the coupling computed in the previous steps, we will generate an histogram for the squared couplings in logarithmic scale,

In [38]:
!cp "${VOTCASHARE}/xtp/xml/ianalyze.xml" OPTIONFILES
edit_calculator("ianalyze", "states", "e,h,s")
!xtp_run -e ianalyze -o OPTIONFILES/ianalyze.xml -f state.hdf5

The option 'states' on file 'OPTIONFILES/ianalyze.xml' has been set to 'e,h,s'
========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... ianalyze
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... ianalyze
 Using 1 threads
Calculating for state e now.
Calculating for state h now.
Calculating for state s now.
Changes have not been written to state file.


## QMMM calculations

In [39]:
!cp qmmm.xml OPTIONFILES/
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
... ... Writing job file qmmm_jobs.xml
... ... In total 3001 jobs
Changes have not been written to state file.


Lets run just the first job

In [40]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_jobs.xml                                                                                                                                                            
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml   
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml  
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -x 2 -f state.hdf5 -j run

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
MST ERR Job file = 'qmmm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_jobs.xml
MST ERR Registered 3000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ...  Regions created
T00 ERR ... Id: 0 type: qm size: 1 charge[e]= 12
T00 ERR ... Id: 1 type: polar size: 92 charge[e]= -5.09315e-15
T00 ERR ... Id: 2

Finally, save the results

In [41]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_parallel, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:29:22)
votca_csg, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:11:42)
votca_tools, version 2022-dev gitid: 60fc73d (compiled Jul  7 2021, 16:08:43)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
Found 1 states of type s
Found 1 states of type n
2998 incomplete jobs found.

Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 
